In [16]:
import numpy as np
from paddleocr import PaddleOCR
import pandas as pd

In [15]:
ocr = PaddleOCR(use_angle_cls=False,lang='en', show_log=False)

In [17]:
data = pd.read_csv("../data/cleaned_data.csv")

In [19]:
data.photo.shape

(892,)

In [29]:
ocr_res = np.empty(data.photo.shape,dtype='O')
for i,img_path in enumerate(data.photo.values):
    if img_path is not np.nan:
        result = ocr.ocr("../data/"+img_path, cls=False, det=True, rec=True)
        txts = [line[1][0] for line in result]
    else:
        txts = ['']
    ocr_res[i] = ' '.join(txts)
    if i%100==0:
        print(f"{i} images done!")



0 images done!
100 images done!
200 images done!
300 images done!
400 images done!
500 images done!
600 images done!
700 images done!
800 images done!


In [33]:
data["ocr_res"] = ocr_res

In [37]:
data.loc[data.ocr_res == '']['type'].value_counts()

job_post    151
others       18
empty        15
link          2
Name: type, dtype: int64

In [38]:
data.to_csv("../data/data_v2.csv",index=False)